In [1]:
from classes import *
from helper_functions import *

import pandas as pd
import numpy as np
import datetime, time
import random
random.seed(2022)

# for configurations
from dotenv import load_dotenv
import sys, os, re, math
load_dotenv()

# for distance matrix
import gmaps
import googlemaps
from haversine import haversine
from scipy.spatial.distance import cdist

# for routing
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
## DATA HANDLING
file_path = 'data/processed_data_3.csv'
plan_date = datetime.date.today() # or you can do datetime.date(year,month,day)
num_vehicles = 5
ins_ends_coords = [(-34.8218243,138.7292797),(-34.8104796,138.6111791),(-34.8938435,138.6918266),(-34.7825552,138.610732),(-34.8516117,138.6722955)]
data = pd.read_csv(file_path) # Reading the csv

In [3]:
def solve(data,plan_date,num_vehicles,ins_ends_coords):
    
    ## CONFIGURATION 
    data = data.copy(deep=True)
    API_KEY = os.getenv("API_KEY") # replace api key with your own here
    FACTORY_GEO_COORD = os.getenv("FACTORY_GEO_COORD") 
    factory_coord = list(map(float,FACTORY_GEO_COORD.split(',')))
    gmaps.configure(api_key=API_KEY)
    
    ## FILTERING ONLY 1 INSTALLER JOBS WITH STATUS = None
    jobs_installer_1 = data[(data["installers_required"]==1) & pd.isnull(data["status"])].reset_index(drop=True)

    ## PROCESSING DATA FOR VRP USE
    df_pending = jobs_installer_1[['id','Latitude','Longitude']]
    new_row = pd.DataFrame({'id':0, 'Latitude':factory_coord[0],'Longitude':factory_coord[1]}, index =[0]) # Inserting factory location to top
    df_pending = pd.concat([new_row, df_pending]).reset_index(drop=True)
    df_pending.set_index('id', inplace=True)
    df_pending['Latitude'] = df_pending['Latitude'].astype(float)
    df_pending['Longitude'] = df_pending['Longitude'].astype(float)

    for i in range(len(ins_ends_coords)):  # Inserting installers' end locations to df_pending
        ins_id = i+1
        df_pending.loc[ins_id] = [ins_ends_coords[i][0],ins_ends_coords[i][1]]
        
    # SETTING UP variables and figures for VISUALISATION
    jobs, ins_ends = [],[]
    for i, row in df_pending.iterrows():
        if i == 0:
            continue
        elif i//100 == 0: # Expecting max 100 installers for the program and job_ids no less than 100
            ins_end = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
            ins_ends.append(ins_end)
        else:
            job = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
            jobs.append(job)

    factory = {'location': (factory_coord[0],factory_coord[1])}
    factory_layer = gmaps.symbol_layer([factory['location']], hover_text='Factory', info_box_content='Factory', fill_color='white', stroke_color='red', scale=6)

    job_locations = [job['location'] for job in jobs]
    job_labels = [job['id'] for job in jobs]
    jobs_layer = gmaps.symbol_layer(
        job_locations, hover_text=job_labels, fill_color='white', stroke_color='black', scale=3
    )

    ins_end_locations = [ins_end['location'] for ins_end in ins_ends]
    ins_labels = [ins_end['id'] for ins_end in ins_ends]
    ins_ends_layer = gmaps.symbol_layer(
        ins_end_locations, hover_text=ins_labels, fill_color='white', stroke_color='red', scale=3
    )

    fig = gmaps.figure()
    fig.add_layer(factory_layer)
    fig.add_layer(jobs_layer)
    fig.add_layer(ins_ends_layer)
    # fig
    
    ## DEFINING PENALTIES AND JOB_DURATIONS(DEMANDS)
    demands, penalties = [0],[0]
    pref_dates, pref_days, pref_installers=[None],[None],[None] # first values for depot 
    pref_time_windows=[None]
    ## DEMANDS 
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'expected_job_time']):
            demands.append(int(60))
            continue
        demands.append(int(jobs_installer_1.loc[i,'expected_job_time']))

    ## DUE_DATE_DIFFERENCE    
    for i in range(len(jobs)):
        due_date = datetime.datetime.strptime(jobs_installer_1.loc[i,'est_installation_date'], '%d/%m/%Y').date()
        curr_date = datetime.date.today()
        due_date_diff = (due_date-curr_date).days
        penalties.append(due_date_diff)

    ## CUSTOMER PREFERRED DATE
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_date']):
            pref_dates.append(None)
            continue
        pref_date_to_append = datetime.datetime.strptime(jobs_installer_1.loc[i,'pref_date'], '%d/%m/%Y').date()
        pref_dates.append(pref_date_to_append)

    ## CUSTOMER PREFERRED DAY
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_day']):
            pref_days.append(None)
            continue
        pref_day_to_append = int(jobs_installer_1.loc[i,'pref_day'])
        pref_days.append(pref_day_to_append)

    ## CUSTOMER PREFERRED INSTALLER
    for i in range(len(jobs)):
        if np.isnan(jobs_installer_1.loc[i,'pref_installer']):
            pref_installers.append(None)
            continue
        pref_installers.append(int(jobs_installer_1.loc[i,'pref_installer']))

    ## CUSTOMER PREFERRED TIME WINDOWS
    for i in range(len(jobs)):
        if pd.isnull(jobs_installer_1.loc[i,'pref_time_window']):
            pref_time_windows.append(None)
            continue
        curr_pref_time_window = jobs_installer_1.loc[i,'pref_time_window'].split(',')
        curr_pref_time_window[0] = time_to_minutes(curr_pref_time_window[0])
        curr_pref_time_window[1] = time_to_minutes(curr_pref_time_window[1])
        pref_time_windows.append(curr_pref_time_window)

    ## END LOCATIONS OF INSTALLERS
    end_locations = []
    for i in range(num_vehicles):
        end_locations.append(len(jobs)+i+1)

    dist_matrix,time_matrix = get_distance_time_matrices(df_pending)
    job_ids = df_pending.index.tolist()
    
    routes, total_distance, total_load, job_times = solve_vrp_for(time_matrix, num_vehicles, demands, penalties, end_locations, pref_dates, pref_days, pref_installers, pref_time_windows, plan_date, job_ids)
    
    ## UNCOMMENT THE CODE BELOW TO VISUALIZE THE ROUTES
    if routes:
        map_solution(factory, jobs, ins_ends, routes, fig)
    else:
        print('No solution found.') 
    display(fig)
    
    if job_times==None or (not job_times):
        pass
    else:
        for job_id, out_values in job_times.items():
            start_time_str = "{:02d}:{:02d}".format((out_values['start_time']//60)+8, out_values['start_time']%60)
            end_time_str = "{:02d}:{:02d}".format((out_values['end_time']//60)+8, out_values['end_time']%60)
            data.loc[data['id'] == job_id, ['installation_date','arrival_start_time','arrival_end_time','installer_ids','status']] = plan_date.strftime("%d/%m/%Y"), start_time_str, end_time_str, str(out_values['installer_id']), 'Scheduled'
    
    return routes, total_distance, total_load, job_times, data

In [4]:
day1 = next_working_date(plan_date)
routes1, total_distance1, total_load1, job_times1, data_output_day1 = solve(data,day1,num_vehicles,ins_ends_coords)

Hooday 2223 0
Hooday 66792 0
Hooday 68603 0
Planning Date: 2023-02-06
Dropped nodes: 2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25,26,28,29,31,32,34,35,36,37,38,39,42,43,47,49,50,51,54,55,57,59,60,62,63,64,65,67,68,69,70,71,72,73,75,76,77,78,79,82,83,84,89,90,91,92,93,94,96,97,98,99,100,101,102,103,104,105,106,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  ID:70307, Time:(13-133) ->  ID:68603, Time:(176-236) ->  ID:71451, Time:(273-333) ->  ID:70920, Time:(349-409) ->  Ins_End 0 @ (416min)
Total time spent: 416min
Total job time spent: 300min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  ID:66769, Time:(29-89) ->  ID:69433, Time:(91-121) ->  ID:66013, Time:(128-158) ->  ID:70420, Time:(172-232) ->  ID:2223, Time:(237-387) ->  ID:71270, Time:(392-452) ->  Ins_End 1 @ (478min)
Total time spent: 478min
Total job time spent: 390min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:69612, Time:(14-74) ->  ID:70822, Time:(88-148) ->  ID:70566, Time:(153-213) ->  ID:71273, Time:(223-283) ->

Figure(layout=FigureLayout(height='420px'))

In [5]:
day2 = next_working_date(day1)
routes2, total_distance2, total_load2, job_times2, data_output_day2 = solve(data_output_day1,day2,num_vehicles,ins_ends_coords)

Hooday 66190 1
Hooday 67170 1
Hooday 70101 1
Hooday 70784 1
Hooday 70903 1
Hooday 71730 1
Planning Date: 2023-02-07
Dropped nodes: 1,2,3,4,5,6,7,8,9,10,11,12,15,17,18,20,21,22,23,24,27,30,33,34,35,36,38,39,40,41,43,45,46,49,50,54,55,56,57,59,61,62,63,65,66,67,69,70,71,72,74,76,77,79,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  ID:67105, Time:(57-147) ->  ID:70591, Time:(173-233) ->  ID:66917, Time:(265-325) ->  ID:2769, Time:(415-421) ->  Ins_End 0 @ (461min)
Total time spent: 461min
Total job time spent: 216min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  ID:70825, Time:(23-143) ->  ID:71374, Time:(145-265) ->  ID:2758, Time:(273-393) ->  ID:71081, Time:(401-461) ->  Ins_End 1 @ (477min)
Total time spent: 477min
Total job time spent: 420min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:70887, Time:(11-71) ->  ID:70886, Time:(106-226) ->  ID:66190, Time:(234-294) ->  ID:66850, Time:(299-359) ->  ID:71654, Time:(379-469) ->  Ins_End 2 @ (476min)
Total time spent: 476min
Total job time sp

Figure(layout=FigureLayout(height='420px'))

In [6]:
day3 = next_working_date(day2)
routes3, total_distance3, total_load3, job_times3, data_output_day3 = solve(data_output_day2,day3,num_vehicles,ins_ends_coords)

Hooday 70681 2
Hooday 71097 2
Hooday 71666 2
Planning Date: 2023-02-08
Dropped nodes: 2,4,5,7,10,11,12,14,15,16,17,20,22,23,26,27,28,29,30,32,33,34,35,36,37,39,41,43,45,46,47,49,50,51,53,54,
Route for vehicle 0:
 ID:0, Time:(0-0) ->  ID:71097, Time:(20-110) ->  ID:71498, Time:(121-151) ->  ID:2521, Time:(156-336) ->  ID:67033, Time:(347-467) ->  Ins_End 0 @ (478min)
Total time spent: 478min
Total job time spent: 420min

Route for vehicle 1:
 ID:0, Time:(0-0) ->  ID:2702, Time:(7-187) ->  ID:71069, Time:(216-456) ->  Ins_End 1 @ (473min)
Total time spent: 473min
Total job time spent: 420min

Route for vehicle 2:
 ID:0, Time:(0-0) ->  ID:2773, Time:(32-152) ->  ID:2608, Time:(166-286) ->  ID:70681, Time:(297-387) ->  ID:71666, Time:(401-461) ->  Ins_End 2 @ (463min)
Total time spent: 463min
Total job time spent: 390min

Route for vehicle 3:
 ID:0, Time:(0-0) ->  ID:2662, Time:(42-162) ->  ID:2363, Time:(175-295) ->  ID:69754, Time:(299-359) ->  ID:69278, Time:(381-441) ->  Ins_End 3 @ (4

Figure(layout=FigureLayout(height='420px'))

In [7]:
# ## GOOGLE DISTANCE MATRIX API
# cumul_dist,cumul_time=0,0
# for route in routes:
#     total_distance,total_time = 0,0
#     prev_node = None
#     curr_node = None
#     for node in routes[route]:
#         if prev_node==None:
#             prev_node = node
#             continue
#         curr_node = node
#         prev_coord = (df_pending.iloc[prev_node]['Latitude'],df_pending.iloc[prev_node]['Longitude'])
#         curr_coord = (df_pending.iloc[curr_node]['Latitude'],df_pending.iloc[curr_node]['Longitude'])
#         total_distance += get_distance(prev_coord,curr_coord)
#         total_time += get_travel_time(prev_coord,curr_coord)
# #         print(prev_coord, end='--')
#         prev_node = curr_node
# #     print(curr_coord)
#     print(total_distance/1000,total_time)
#     cumul_dist+=total_distance
#     cumul_time+=total_time
# print(cumul_dist/1000,cumul_time)

In [10]:
data_output_day3.to_csv('output/processed_data_3_out.csv',index=False)